### Questions
* Subqueries - let's go more into them
* Many to Many Joins
* Noticed that halfway through the lesson they used column parsing

### Objectives
YWBAT 
- perform various queries on a sqlite db
- build functions to perform queries
- perform join queries

### Outline
* Questions
* Load in some data and do some queries
* Build some functions to make our lives easier and foreshadow tomorrow's work
* Do some join queries
* perhaps some subqiueries
* wrap up

In [1]:
import pandas as pd
import numpy as np

import sqlite3

import matplotlib.pyplot as plt

In [2]:
conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()

In [3]:
def load_df(table_name=None, conn=None):
    query = 'select * from {}'.format(table_name)
    df = pd.read_sql(query, conn)
    return df

In [5]:
# table_names to be a list of my table_names
table_names = [res[0] for res in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()]
table_names

['orderdetails',
 'payments',
 'offices',
 'customers',
 'orders',
 'productlines',
 'products',
 'employees',
 'contacts',
 'contacts2']

### A throwback favorite 

In [6]:
d = {} # table_name: dataframe of table
for table_name in table_names:
    d[table_name] = load_df(table_name, conn)

In [17]:
orderdetails_df = load_df('orderdetails', conn)
orderdetails_df.head()

orderNumber productCode quantityOrdered priceEach orderLineNumber
0       10100    S18_1749              30    136.00               3
1       10100    S18_2248              50     55.09               2
2       10100    S18_4409              22     75.46               4
3       10100    S24_3969              49     35.29               1
4       10101    S18_2325              25    108.06               4

In [18]:
products_df = load_df('products', conn)
products_df.head()

productCode                            productName   productLine  \
0    S10_1678  1969 Harley Davidson Ultimate Chopper   Motorcycles   
1    S10_1949               1952 Alpine Renault 1300  Classic Cars   
2    S10_2016                  1996 Moto Guzzi 1100i   Motorcycles   
3    S10_4698   2003 Harley-Davidson Eagle Drag Bike   Motorcycles   
4    S10_4757                    1972 Alfa Romeo GTA  Classic Cars   

  productScale             productVendor  \
0         1:10           Min Lin Diecast   
1         1:10   Classic Metal Creations   
2         1:10  Highway 66 Mini Classics   
3         1:10         Red Start Diecast   
4         1:10   Motor City Art Classics   

                                  productDescription quantityInStock buyPrice  \
0  This replica features working kickstand, front...            7933    48.81   
1  Turnable front wheels; steering function; deta...            7305    98.58   
2  Official Moto Guzzi logos and insignias, saddl...            6625    68.99   
3  Model features, official Harley Davidson logos...            5582    91.02   
4  Features include: Turnable front wheels; steer...            3252    85.68   

     MSRP  
0   95.70  
1  214.30  
2  118.94  
3  193.66  
4  136.00

In [21]:
query = 'select * from customers;'
customers_df = pd.read_sql(query, conn)
customers_df.head()

customerNumber                customerName contactLastName contactFirstName  \
0            103           Atelier graphique         Schmitt          Carine    
1            112          Signal Gift Stores            King             Jean   
2            114  Australian Collectors, Co.        Ferguson            Peter   
3            119           La Rochelle Gifts         Labrune          Janine    
4            121          Baane Mini Imports      Bergulfsen           Jonas    

          phone                  addressLine1 addressLine2       city  \
0    40.32.2555                54, rue Royale                  Nantes   
1    7025551838               8489 Strong St.               Las Vegas   
2  03 9520 4555             636 St Kilda Road      Level 3  Melbourne   
3    40.67.8555  67, rue des Cinquante Otages                  Nantes   
4    07-98 9555        Erling Skakkes gate 78                 Stavern   

      state postalCode    country salesRepEmployeeNumber creditLimit  
0                44000     France                   1370    21000.00  
1        NV      83030        USA                   1166    71800.00  
2  Victoria       3004  Australia                   1611   117300.00  
3                44000     France                   1370   118200.00  
4                 4110     Norway                   1504    81700.00

### Using PRAGMA helps identify primary key

In [40]:
cursor.execute("PRAGMA table_info(contacts2);").fetchall()

[(0, 'contact_id', 'INTEGER', 1, None, 0),
 (1, 'first_name', 'TEXT', 1, None, 0),
 (2, 'last_name', 'TEXT', 1, None, 0),
 (3, 'email', 'text', 1, None, 0),
 (4, 'phone', 'text', 1, None, 0)]

### WHY SHOULD YOU ALWAYS SPECIFY YOUR PRIMARY KEYS!?

SQL will get super mad if we try and produce duplicates. It keeps us safe. 

In [42]:
customers_df.head(2)

customerNumber        customerName contactLastName contactFirstName  \
0            103   Atelier graphique         Schmitt          Carine    
1            112  Signal Gift Stores            King             Jean   

        phone     addressLine1 addressLine2       city state postalCode  \
0  40.32.2555   54, rue Royale                  Nantes            44000   
1  7025551838  8489 Strong St.               Las Vegas    NV      83030   

  country salesRepEmployeeNumber creditLimit  
0  France                   1370    21000.00  
1     USA                   1166    71800.00

In [41]:
orders_df = pd.read_sql("select * from orders", conn)
orders_df.head()

orderNumber   orderDate requiredDate shippedDate   status  \
0       10100  2003-01-06   2003-01-13  2003-01-10  Shipped   
1       10101  2003-01-09   2003-01-18  2003-01-11  Shipped   
2       10102  2003-01-10   2003-01-18  2003-01-14  Shipped   
3       10103  2003-01-29   2003-02-07  2003-02-02  Shipped   
4       10104  2003-01-31   2003-02-09  2003-02-01  Shipped   

                 comments customerNumber  
0                                    363  
1  Check on availability.            128  
2                                    181  
3                                    121  
4                                    141

In [47]:
orders_df.shape, join_1.shape

((326, 7), (326, 4))

In [44]:
# let's do a join on customer number
# customerName, customer.phone,  customer state, customer order-date
query = """
        SELECT c.customerName, c.phone, c.state, o.orderDate FROM
        customers as c
        JOIN orders as o using (customerNumber);
        """

In [46]:
join_1 = pd.read_sql(query, conn)
join_1.head()

customerName       phone state   orderDate
0   Atelier graphique  40.32.2555        2003-05-20
1   Atelier graphique  40.32.2555        2004-09-27
2   Atelier graphique  40.32.2555        2004-11-25
3  Signal Gift Stores  7025551838    NV  2003-05-21
4  Signal Gift Stores  7025551838    NV  2004-08-06

### Level Up: Display the names of each product each employee has sold

In [7]:
employees_df = pd.read_sql("select * from employees", conn)
employees_df.head(1)

employeeNumber lastName firstName extension                         email  \
0           1002   Murphy     Diane     x5800  dmurphy@classicmodelcars.com   

  officeCode reportsTo   jobTitle  
0          1            President

In [8]:
customers_df.head(1)

NameError: name 'customers_df' is not defined

In [66]:
query1 = """
        create temporary table emp_cust as
        select e.lastName, e.firstName, c.salesRepEmployeeNumber, c.customerNumber
        from employees as e
        join customers as c 
        on e.employeeNumber = c.salesRepEmployeeNumber;
        """

query2 = """
         create temporary table eco as 
         select ec.customerNumber, ec.lastName, ec.firstName, o.orderNumber
         from emp_cust as ec
         join orders as o
         using (customerNumber);"""


# cursor.execute(query1).fetchall()
cursor.execute(query2).fetchall()

[]

In [73]:
query3 = """
         create temporary table ecod as
         select eco.orderNumber, eco.lastName, eco.firstName, eco.orderNumber, od.productCode
         from eco
         join orderdetails as od
         using (orderNumber);"""
# cursor.execute(query3).fetchall()

In [51]:
orders_df.head(1)

orderNumber   orderDate requiredDate shippedDate   status comments  \
0       10100  2003-01-06   2003-01-13  2003-01-10  Shipped            

  customerNumber  
0            363

In [85]:
query = 'select * from eco;'
cursor.execute(query).fetchall()[:3]

[('124', 'Jennings', 'Leslie', '10113'),
 ('124', 'Jennings', 'Leslie', '10135'),
 ('124', 'Jennings', 'Leslie', '10142')]

In [52]:
orderdetails_df.head(1)

orderNumber productCode quantityOrdered priceEach orderLineNumber
0       10100    S18_1749              30    136.00               3

In [84]:
query = """
        select eco.firstName, eco.lastName, eco.productCode, p.productName
        from ecod as eco
        join products as p
        using (productCode)
        group by eco.firstName, eco.lastName, p.productName
        order by eco.firstName, p.productName"""
cursor.execute(query).fetchall()[:3]

[('Andy', 'Fixter', 'S18_3136', '18th Century Vintage Horse Carriage'),
 ('Andy', 'Fixter', 'S24_2841', '1900s Vintage Bi-Plane'),
 ('Andy', 'Fixter', 'S24_4278', '1900s Vintage Tri-Plane')]

In [53]:
products_df.head(1)

productCode                            productName  productLine  \
0    S10_1678  1969 Harley Davidson Ultimate Chopper  Motorcycles   

  productScale    productVendor  \
0         1:10  Min Lin Diecast   

                                  productDescription quantityInStock buyPrice  \
0  This replica features working kickstand, front...            7933    48.81   

    MSRP  
0  95.70

### What did you learn?
* PRAGMA
* Making long strings
* Temporary tables
* pd.read_sql
* dictionary key:value -> tablename: dataframes

# Subquery Section

In [24]:
### let's look at employee customer tables first
employees_df.head(1)

employeeNumber lastName firstName extension                         email  \
0           1002   Murphy     Diane     x5800  dmurphy@classicmodelcars.com   

  officeCode reportsTo   jobTitle  
0          1            President

In [26]:
customers_df = pd.read_sql("select * from customers;", conn)
customers_df.head(1)

customerNumber       customerName contactLastName contactFirstName  \
0            103  Atelier graphique         Schmitt          Carine    

        phone    addressLine1 addressLine2    city state postalCode country  \
0  40.32.2555  54, rue Royale               Nantes            44000  France   

  salesRepEmployeeNumber creditLimit  
0                   1370    21000.00

# Query 0

In [30]:
query = """
        select e.firstname, e.lastname, e.employeenumber, c.customerNumber
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber;"""

pd.read_sql(query, conn).head(2)

firstName  lastName employeeNumber customerNumber
0    Leslie  Jennings           1165            124
1    Leslie  Jennings           1165            129

In [29]:
# Now let's take this and incorporate the next table orders
orders_df.head(1)

orderNumber   orderDate requiredDate shippedDate   status comments  \
0       10100  2003-01-06   2003-01-13  2003-01-10  Shipped            

  customerNumber  
0            363

# Query 1

In [39]:
# add orders to previous queries joining on customernumber
query = """
        select e.firstname, e.lastname, e.employeenumber, c.customerNumber, o.orderNumber
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber
        join orders as o
        on o.customerNumber = c.customerNumber;"""

pd.read_sql(query, conn).head(2)

firstName  lastName employeeNumber customerNumber orderNumber
0    Leslie  Jennings           1165            124       10113
1    Leslie  Jennings           1165            124       10135

In [40]:
# now let's incororate the orderdetails table
orderdetails_df.head(1)

orderNumber productCode quantityOrdered priceEach orderLineNumber
0       10100    S18_1749              30    136.00               3

# Query 2

In [42]:
# adding product code, removing employee number and customer number
query = """
        select e.firstname, e.lastname, o.orderNumber, od.productCode
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber
        join orders as o
        on o.customerNumber = c.customerNumber
        join orderdetails as od
        on od.orderNumber = o.orderNumber;"""

pd.read_sql(query, conn).head(2)

firstName  lastName orderNumber productCode
0    Leslie  Jennings       10113    S12_1666
1    Leslie  Jennings       10113    S18_1097

In [43]:
# now let's investigate the product table
pd.read_sql("select * from products", conn).head(1)

productCode                            productName  productLine  \
0    S10_1678  1969 Harley Davidson Ultimate Chopper  Motorcycles   

  productScale    productVendor  \
0         1:10  Min Lin Diecast   

                                  productDescription quantityInStock buyPrice  \
0  This replica features working kickstand, front...            7933    48.81   

    MSRP  
0  95.70

# Query 3 - Final

In [45]:
# Now let's add product name with some final group by to remove duplicates
# removing order number and product code
query = """
        select e.firstname, e.lastname, p.productName
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber
        join orders as o
        on o.customerNumber = c.customerNumber
        join orderdetails as od
        on od.orderNumber = o.orderNumber
        join products as p
        on p.productCode = od.productCode;"""

pd.read_sql(query, conn).head(2)

firstName  lastName             productName
0    Leslie  Jennings          1958 Setra Bus
1    Leslie  Jennings  1940 Ford Pickup Truck

### Great! It's working. Let's add some group bys for duplicates!

In [52]:
# alias the columns and add group by at the bottom
query = """
        select e.firstname as fn, e.lastname as ln, p.productName as pn
        from employees as e
        join customers as c
        on e.employeenumber = c.salesrepemployeenumber
        join orders as o
        on o.customerNumber = c.customerNumber
        join orderdetails as od
        on od.orderNumber = o.orderNumber
        join products as p
        on p.productCode = od.productCode
        group by fn, ln, pn;"""

final_df = pd.read_sql(query, conn)
final_df.head(2)

fn      ln                                   pn
0  Andy  Fixter  18th Century Vintage Horse Carriage
1  Andy  Fixter               1900s Vintage Bi-Plane

In [57]:
final_df.shape, final_df.drop_duplicates().shape

((1368, 3), (1368, 3))